# Initial setting

## libraries

In [2]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get(
    "https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css"
).text
HTML(styles)

In [61]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import functools

from scipy import stats

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

%matplotlib inline

import seaborn as sns
sns.set(style='whitegrid')
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

In [4]:
# create a progressbar function
def progressbar(n_step, n_total):
    """Prints self-updating progress bar to stdout to track for-loop progress
    
    There are entire 3rd-party libraries dedicated to custom progress-bars.
    A simple function like this is often more than enough to get the job done.
    
    :param n_total: total number of expected for-loop iterations
    :type n_total: int
    :param n_step: current iteration number, starting at 0
    :type n_step: int

    .. example::
    
        for i in range(n_iterations):
            progressbar(i, n_iterations)
            
    .. source:
    
        This function is a simplified version of code found here:
        https://stackoverflow.com/questions/3160699/python-progress-bar/15860757#15860757
    """
    n_step = n_step + 1
    barlen = 50
    progress = n_step / n_total
    block = int(round(barlen * progress))
    status = ""
    if n_step == n_total:
        status = "Done...\r\n\n"
    text = "\r [{0}] {1}/{2} {3}".format(
        "=" * block + "-" * (barlen - block),
        n_step,
        n_total,
        status,
    )
    sys.stdout.write(text)
    sys.stdout.flush()

## dataset

In [5]:
# Load data
df = pd.read_csv('data/df_fifa.csv')

# Data handling

## create the dataset for Part C

In [6]:
# select players in the following clubs as test set
df_c = df[df['d_div1_league']==1]

# extract league and club
df_league_club = df_c.copy()
df_league_club = df_league_club[['league_name','club']].drop_duplicates()
df_league_club['variable'] = 'club_' + df_league_club['club']

# create lead variables for overall and value_eur (overall and value scores in the following year)
df_c = df_c.sort_values(['sofifa_id','year'])
df_c.insert(9,'overall_next',df_c.groupby(['sofifa_id'])['overall'].shift(-1))
df_c.insert(11,'value_eur_next',df_c.groupby(['sofifa_id'])['value_eur'].shift(-1))

# create dummies for club, nationality, 
df_c_dummies = pd.get_dummies(df_c[['nationality','club','work_rate']])
df_c = pd.concat([df_c, df_c_dummies], axis=1)

## drop columns, imputation

In [7]:
df_c.to_csv('data/df_c.csv', index=False)

In [8]:
# drop unnecessary columns
drop_vars = ['sofifa_id','short_name','dob','nationality','club','wage_eur','preferred_foot','work_rate','body_type',
             'team_position','team_jersey_number','joined','contract_valid_until','league_name','release_clause_eur',
             'loaned_from','main_position','year']
df_c_all = df_c.drop(drop_vars, axis=1)

# impute zero values for fieldplayers and goalkeeping ability
# this is because goalkeeping ability is not available for field players and vice versa
impute_vars = ['ab_pace','ab_shooting','ab_passing','ab_dribbling','ab_defending','ab_physic'] + [x for x in df_c_all.columns if x.startswith('ab_gk')]
for var in impute_vars:
    df_c_all[var] = df_c_all[var].fillna(0)
    
# impute mean value for ab_mentality_conposure
# Composure is a Player Attribute in FIFA that determines a player's the state or feeling of being calm and 
# controlling their frustration in matches frustration. (from FIFAplay)
df_c_all['ab_mentality_composure'].fillna(df_c_all['ab_mentality_composure'].mean(), inplace=True)

# save to csv
df_c_all.to_csv('data/df_c_all.csv', index=False)

# Regression - Overall score

Basic idea  
- We define a club's ability to increase a player stats and value as the club's performance when we control the player's basic characteristics, e.g. age, reputation, skills, etc. 
- Given this, we regress players' overall score (annual change) on all these characteristics and club dummies, and identify the feature importance of a club dummy as the club's performance when controlling other factors.

## Filter data, standardization, etc.

In [9]:
# separate a dataset for overall score regression and drop NA
df_c_overall = df_c_all.copy().drop(['value_eur', 'value_eur_next'], axis=1)
df_c_overall = df_c_overall.dropna()
df_c_overall['diff_overall'] = df_c_overall['overall_next'] - df_c_overall['overall'] 

# save to csv
df_c_overall.to_csv('data/df_c_overall.csv', index=False)
df_c_overall.shape

(10517, 357)

In [10]:
# assign X and y
X = df_c_overall.drop(['overall_next','overall','diff_overall'], axis=1)
y = df_c_overall['diff_overall']

## standardization
#scaler = StandardScaler().fit(X)
#X_stan = scaler.transform(X)

## LASSO

In [100]:
# grid search
# set parameters
la_alphas = [1e-2, 1e-1, 1, 1e+1]

# create empty lists to store errors
la_tr_err, la_val_err = [],[]

# run regression for each alpha
for i,alpha in enumerate(la_alphas):
    # update progressbar
    progressbar(i, len(la_alphas))
    
    # perform cross-validation on the training data with 10 folds and get the mse_scores
    lasso = Lasso(alpha=alpha, max_iter=10000)
    scores = cross_validate(lasso, X, y, 
                            cv=5, 
                            scoring='neg_mean_squared_error', 
                            return_train_score=True,
                            n_jobs=-1)
    
    #Compute the train and validation MSE
    la_tr_err.append(scores['train_score'].mean() * -1)
    la_val_err.append(scores['test_score'].mean() * -1)

# find the degree that returns the minimum validation error
la_min_val_err = min(la_val_err)
la_best_alpha = la_alphas[la_val_err.index(la_min_val_err)]
print(la_min_val_err, la_best_alpha)

 [==================================================] 5/5 Done...

6.797436762426119 0.01


In [104]:
# refit Lasso using best alpha
la_best = Lasso(alpha=la_best_alpha, max_iter=10000)
la_best.fit(X, y)

Lasso(alpha=0.01, max_iter=10000)

In [105]:
# store coefficients into dataframe
df_la_fi = pd.DataFrame({'variable':X.columns,
                         'overall_lasso':la_best.coef_})
df_la_fi = df_la_fi[df_la_fi['variable'].str.startswith('club')]

In [23]:
df_la_fi.to_csv('data/df_la_fi.csv', index=False)

## Random Forest (m=p/3)

In [108]:
# parameters
rf_trees = list(range(100, 120, 20))
rf_depths = list(range(9, 10, 1))
 
rf_params = {'n_estimators': rf_trees, 
             'max_depth': rf_depths}
 
# grid search
rf = RandomForestRegressor(warm_start=True,max_features=int(X_stan.shape[1]/3),random_state=0)
rf_gs = GridSearchCV(estimator=rf,param_grid=rf_params,scoring='neg_mean_squared_error',verbose=1,n_jobs=-1)
rf_gs.fit(X, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   10.3s remaining:   15.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   10.5s finished


GridSearchCV(estimator=RandomForestRegressor(max_features=118, random_state=0,
                                             warm_start=True),
             n_jobs=-1, param_grid={'max_depth': [9], 'n_estimators': [100]},
             scoring='neg_mean_squared_error', verbose=1)

In [109]:
# extract best parameters and estimator
rf_best_param = rf_gs.best_params_
rf_best_estimator = rf_gs.best_estimator_

In [117]:
# select club dummies
clubs = [x for x in X.columns if x.startswith('club')]

# compute feature importance
# for each club, compute the difference of predicted values when the club dummy = 1 and 0
# impute mode values for all variables except for club dummies. For other clubs, impute 0
X_rf_fi = pd.DataFrame(data=X,
                       columns=X.columns)

# all club dummies = 0 
X_rf_fi_zero = X_rf_fi.copy()
X_rf_fi_zero[clubs] = 0

rf_fi = []
for club in clubs:
    
    # dummy for the seleted club = 1
    X_rf_fi_one = X_rf_fi_zero.copy()
    X_rf_fi_one[club] = 1
    
    # predict for each dataframe and take difference
    pred_one = rf_best_estimator.predict(X_rf_fi_one).mean()
    pred_zero = rf_best_estimator.predict(X_rf_fi_zero).mean()
    pred_diff = pred_one - pred_zero
    
    # append
    rf_fi.append(pred_diff)

In [118]:
# store coefficients into dataframe
df_rf_fi = pd.DataFrame({'variable':clubs,
                         'overall_rf':rf_fi})
df_rf_fi

,variable,overall_rf
0,club_1. FC Köln,0.000337
1,club_1. FC Union Berlin,-0.003157
2,club_1. FSV Mainz 05,0.016044
3,club_AS Monaco,0.019379
4,club_AS Saint-Étienne,-0.000780
...,...,...
93,club_VfL Wolfsburg,0.006426
94,club_Villarreal CF,0.008194
95,club_Watford,0.001333
96,club_West Ham United,-0.005608


In [119]:
# merge all dataframes
merge_dfs = [df_la_fi, df_rf_fi]
df_fi = functools.reduce(lambda  left,right: pd.merge(left,right,on=['variable']), merge_dfs)
df_fi['ovr_la_scaled'] = MinMaxScaler().fit_transform(df_fi[['overall_lasso']]) * 100
df_fi['ovr_rf_scaled'] = MinMaxScaler().fit_transform(df_fi[['overall_rf']]) * 100
df_fi['mean'] = (df_fi['ovr_la_scaled'] + df_fi['ovr_rf_scaled']) / 2
df_fi = df_fi.sort_values('mean', ascending=False)
df_fi

,variable,overall_lasso,overall_rf,ovr_la_scaled,ovr_rf_scaled,mean
5,club_Amiens SC,0.000000,0.303565,73.161477,100.000000,86.580738
9,club_Atalanta,0.056049,0.128313,100.000000,53.591704,76.795852
64,club_RB Leipzig,0.000000,0.109768,73.161477,48.680673,60.921075
67,club_RCD Espanyol,0.000000,0.103920,73.161477,47.132105,60.146791
10,club_Athletic Club de Bilbao,0.000000,0.090461,73.161477,43.568135,58.364806
...,...,...,...,...,...,...
52,club_Manchester United,-0.000000,-0.037930,73.161477,9.568935,41.365206
61,club_Olympique de Marseille,-0.000000,-0.051672,73.161477,5.929678,39.545577
86,club_Stade de Reims,-0.000000,-0.074065,73.161477,0.000000,36.580738
20,club_CA Osasuna,-0.044725,-0.039097,51.745420,9.259812,30.502616


In [120]:
df_fi.to_csv('data/df_fi.csv', index=False)

# Visualization

To do  
- Overall score distribution for each club (kernel density by year, 4 row * 5 column, 5 different figures)
- Change in overall score distribution for each club (same)
- Visualization of yearly change in club score
- Visualization of top 5 clubs in club score on map?

In [25]:
# compile the list of dataframes to merge
# merge all dataframes
merge_dfs = [df_lasso_pi, df_en_pi, df_rf_pi]
df_pi = functools.reduce(lambda  left,right: pd.merge(left,right,on=['variable']), merge_dfs)
df_pi

,variable,lasso,elastic_net,random_forest
0,age,0.705445,0.611701,1.373173e+00
1,ab_movement_reactions,0.490214,0.453141,9.267627e-01
2,ab_skill_ball_control,0.351721,0.092524,2.406374e-01
3,d_pos_GK,0.343786,0.026946,8.385480e-07
4,ab_attacking_short_passing,0.249362,0.116767,1.314398e-01
...,...,...,...,...
349,nationality_Curacao,-0.000040,0.000037,0.000000e+00
350,nationality_Northern Ireland,-0.000049,0.000103,5.404835e-04
351,nationality_Austria,-0.000095,0.000182,7.595143e-04
352,club_RC Celta,-0.000122,0.000467,4.348474e-03


In [62]:
# merge all dataset
df_pi_club = df_pi.copy()
df_pi_club = df_pi_club.loc[df_pi_club['variable'].str.startswith('club')]

# rescale all feature importance
models = ['lasso', 'elastic_net', 'random_forest']
models_rescale = []
for model in models:
    df_pi_club[model + '_rescale'] = MinMaxScaler().fit_transform(df_pi_club[[model]])
    models_rescale.append(model + '_rescale')

# calculate mean of all scores 
df_pi_club['mean'] = df_pi_club[models_rescale].mean(axis=1) * 100

# merge league name
df_pi_club = df_pi_club.merge(df_league_club, on='variable')
df_pi_club = df_pi_club.sort_values(['league_name','mean'], ascending=False)
df_pi_club.to_csv('data/df_pi_club.csv', index=False)
df_pi_club

,variable,lasso,elastic_net,random_forest,lasso_rescale,elastic_net_rescale,random_forest_rescale,mean,league_name,club
1,club_Granada CF,0.028969,0.024427,0.002807,0.943408,0.908246,0.052418,63.469052,Spain Primera Division,Granada CF
3,club_CA Osasuna,0.018003,0.015728,0.003405,0.588794,0.584975,0.063962,41.257710,Spain Primera Division,CA Osasuna
9,club_Real Madrid,0.015589,0.014845,0.002090,0.510763,0.552171,0.038559,36.716405,Spain Primera Division,Real Madrid
10,club_RCD Espanyol,0.012973,0.011602,0.009620,0.426143,0.431626,0.184065,34.727800,Spain Primera Division,RCD Espanyol
25,club_Sevilla FC,0.005108,0.003815,0.006933,0.171843,0.142250,0.132138,14.874384,Spain Primera Division,Sevilla FC
...,...,...,...,...,...,...,...,...,...,...
92,club_Burnley,0.000000,0.000000,0.003178,0.006650,0.000468,0.059577,2.223198,English Premier League,Burnley
93,club_Everton,0.000000,0.000000,0.002795,0.006650,0.000468,0.052184,1.976772,English Premier League,Everton
86,club_Manchester United,0.000000,0.000000,0.002304,0.006650,0.000468,0.042704,1.660760,English Premier League,Manchester United
87,club_Liverpool,0.000000,0.000000,0.002138,0.006650,0.000468,0.039481,1.553322,English Premier League,Liverpool
